In [1]:
import probtorch
import torch

import combinators
import smc
import ssm
import utils

In [2]:
generative = smc.ParticleTrace(1)

In [3]:
mu = torch.zeros(1, 1)
sigma = torch.ones(1, 1) / 2
delta = torch.ones(1, 1)
zs = torch.ones(1, 50+1) * -1
zs[:, 0] = 0

In [4]:
for t in range(zs.shape[1] - 1):
    zs[:, t+1], mu, sigma, delta, generative = ssm.ssm_step(zs[:, t], mu, sigma, delta, t, generative)

In [5]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1x1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1x1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1x1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1x1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1x1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1x1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1x1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1x1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1x1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1x1]), 'Z_11': Normal([torch.FloatTenso

In [6]:
num_particles = 500

In [7]:
smc_runner = smc.smc(ssm.ssm_step, ssm.ssm_retrace)

In [8]:
ssm_params = {
    'mu': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'sigma': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'delta': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    }
}

In [9]:
inference, init_ssm = smc.variational_smc(num_particles, ssm.init_ssm, smc_runner, 500, 50, ssm_params, generative)

In [10]:
inference

Trace{'mu': Normal([torch.cuda.FloatTensor of size 500]), 'sigma': Normal([torch.cuda.FloatTensor of size 500]), 'delta': Normal([torch.cuda.FloatTensor of size 500]), 'Z_0': Normal([torch.cuda.FloatTensor of size 500]), 'Z_1': Normal([torch.cuda.FloatTensor of size 500]), 'X_1': Normal([torch.cuda.FloatTensor of size 500x1x1]), 'Z_2': Normal([torch.cuda.FloatTensor of size 500]), 'X_2': Normal([torch.cuda.FloatTensor of size 500x1x1]), 'Z_3': Normal([torch.cuda.FloatTensor of size 500]), 'X_3': Normal([torch.cuda.FloatTensor of size 500x1x1]), 'Z_4': Normal([torch.cuda.FloatTensor of size 500]), 'X_4': Normal([torch.cuda.FloatTensor of size 500x1x1]), 'Z_5': Normal([torch.cuda.FloatTensor of size 500]), 'X_5': Normal([torch.cuda.FloatTensor of size 500x1x1]), 'Z_6': Normal([torch.cuda.FloatTensor of size 500]), 'X_6': Normal([torch.cuda.FloatTensor of size 500x1x1]), 'Z_7': Normal([torch.cuda.FloatTensor of size 500]), 'X_7': Normal([torch.cuda.FloatTensor of size 500x1x1]), 'Z_8': No

In [11]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1x1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1x1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1x1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1x1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1x1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1x1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1x1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1x1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1x1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1x1]), 'Z_11': Normal([torch.FloatTenso

In [12]:
for t in range(1, zs.shape[1]):
    key = 'Z_%d' % t
    accuracy = ((inference[key].value - utils.optional_to(generative[key], inference[key].value))**2).mean()
    print('SMC MSE at time %d: %f' % (t, accuracy))

SMC MSE at time 1: 0.879748
SMC MSE at time 2: 0.197309
SMC MSE at time 3: 5.447684
SMC MSE at time 4: 0.432873
SMC MSE at time 5: 1.063342
SMC MSE at time 6: 2.387942
SMC MSE at time 7: 0.725969
SMC MSE at time 8: 1.031265
SMC MSE at time 9: 0.855301
SMC MSE at time 10: 1.389647
SMC MSE at time 11: 0.128377
SMC MSE at time 12: 1.045317
SMC MSE at time 13: 0.087704
SMC MSE at time 14: 0.708137
SMC MSE at time 15: 0.705568
SMC MSE at time 16: 0.301687
SMC MSE at time 17: 0.145586
SMC MSE at time 18: 0.347885
SMC MSE at time 19: 0.550593
SMC MSE at time 20: 0.295335
SMC MSE at time 21: 0.661778
SMC MSE at time 22: 1.294861
SMC MSE at time 23: 0.759832
SMC MSE at time 24: 0.517229
SMC MSE at time 25: 0.592529
SMC MSE at time 26: 0.690473
SMC MSE at time 27: 0.583936
SMC MSE at time 28: 0.827036
SMC MSE at time 29: 2.549140
SMC MSE at time 30: 2.535645
SMC MSE at time 31: 3.780965
SMC MSE at time 32: 1.375670
SMC MSE at time 33: 0.506408
SMC MSE at time 34: 2.455970
SMC MSE at time 35: 1.1